In [1]:
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup as soup
from newspaper import Article
from time import sleep
import time
import pandas as pd
import numpy as np

1.검색어

In [10]:
qry = '가계부채'
qry1 = quote_plus(qry)

2.url

In [11]:
url0 = "https://search.naver.com/search.naver?&where=news&query={}&start={}"

In [29]:
articles = []
title = []
href = []
date = []
text = []
stime = time.time()
# original 주소 
url0 = "https://search.naver.com/search.naver?&where=news&query={}&start={}"
# 네이버 검색결과 page별
for i in range(3): 
    start = i*10+1
    url = url0.format(qry1,start)
    obj = soup(urlopen(url),'html.parser')
    # links : naver 검색결과 뉴스들의 링크들
    links = obj.find_all('a',{'class':'_sp_each_title'})
    # 링크별 기사가져오기 
    for link in links:
        h = link.get('href') # hyperlink
        a = Article(h,language='ko') # 기사가져오기 
        a.download();a.parse()
        articles.append(a)
        title.append(a.title)
        date.append(a.publish_date)
        text.append(a.text)
        sleep(np.random.randint(3,30))
etime = time.time() - stime
elaps = time.strftime('%H:%M:%S',time.gmtime(etime))
news = pd.DataFrame(dict(qry=qry,date=date,title=title,text=text))        
print('{} search done: {} elapsed'.format(qry,elaps))

가계부채 search done: 00:07:38 elapsed


**naver_search** function

In [30]:
# naver news 검색 
def naver_search(qry,page=3):
    articles = []
    title = []
    href = []
    date = []
    text = []
    stime = time.time()
    # original 주소 
    url0 = "https://search.naver.com/search.naver?where=news&query={}&ds={}&de={}"
    url0 = "https://search.naver.com/search.naver?&where=news&query={}&start={}"
    # 네이버 검색결과 page별
    for i in range(page): 
        start = i*10+1
        qry1 = quote_plus(qry)
        url = url0.format(qry1,start)
        obj = soup(urlopen(url),'html.parser')
        # links : naver 검색결과 뉴스들의 링크들
        links = obj.find_all('a',{'class':'_sp_each_title'})
        # 링크별 기사가져오기 
        for link in links:
            h = link.get('href') # hyperlink
            a = Article(h,language='ko') # 기사가져오기 
            a.download();a.parse()
            articles.append(a)
            title.append(a.title)
            date.append(a.publish_date)
            text.append(a.text)
            sleep(np.random.randint(1,20))
    etime = time.time() - stime
    elaps = time.strftime('%H:%M:%S',time.gmtime(etime))
    news = pd.DataFrame(dict(qry=qry,date=date,title=title,text=text))        
    print('{} search done: {} elapsed'.format(qry,elaps))
    return news

**naver_search_d** function

In [19]:
beg = '2010.01.01'
beg.replace('.','')

'20100101'

In [37]:
# naver news 기간검색
def naver_search_d(qry,beg,end,pages=3): 
    articles = []
    title = []
    href = []
    date = []
    text = []
    stime = time.time()
    # original 주소 
    url0 = "https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={1}&de={2}&docid=&nso=so:r,p:from{3}to{4},a:all&mynews=0&cluster_rank=22&start={5}&refresh_start=0"
    # 네이버 검색결과 page별
    begx = beg.replace('.','')
    endx = end.replace('.','')
    for i in range(pages): 
        start = i*10+1
        qry1 = quote_plus(qry)
        url = url0.format(qry1,beg,end,begx,endx,start)
        obj = soup(urlopen(url),'html.parser')
        # links : naver 검색결과 뉴스들의 링크들
        links = obj.find_all('a',{'class':'_sp_each_title'})
        # 링크별 기사가져오기 
        for link in links:
            h = link.get('href') # hyperlink
            try:
                a = Article(h,language='ko') # 기사가져오기 
                a.download();a.parse()
                articles.append(a)
                title.append(a.title)
                date.append(a.publish_date)
                text.append(a.text)
            except:
                pass
            sleep(np.random.randint(1,7))
    etime = time.time() - stime
    elaps = time.strftime('%H:%M:%S',time.gmtime(etime))
    news = pd.DataFrame(dict(qry=qry,date=date,title=title,text=text))        
    print('{} search done: {} elapsed'.format(qry,elaps))
    return news
# naver_search_d

In [ ]:
news2010 = naver_search_d('가계부채','2010.01.01','2010.12.31',pages=1)

In [41]:
news2010.tail()

,qry,date,title,text
4,가계부채,2010-12-29 12:00:00+09:00,"재정부 ""가계부채 관리가능한 수준""",기획재정부는 29일 가계부채 수준이 비교적 높지만 가계부채가 고소득층에 집중돼 있는...
5,가계부채,None,"부채가구 92% ""1년 후 상환능력 있어""…가계부채 인식 양호",[뉴시스 이시간 핫 뉴스]\n\n【서울=뉴시스】류난영 기자 = 우리나라의 부채가구가...
6,가계부채,2010-12-23 09:23:52+09:00,"김종창 ""가계부채, PF대출, 외국자본 유출입에 능동적 대응""","김종창 금융감독원장은 23일 ""앞으로도 가계부채, PF대출, 외국자본의 유출입 등 ..."
7,가계부채,2010-12-22 13:28:51,"진동수 ""가계부채, 문제는 증가 속도""",박민규 기자 yushin@\n\n\n\n\n\n[아시아경제 박민규 기자] 진동수 금...
8,가계부채,None,"기업銀 '부채 클리닉 서비스', ""가계 빚 고민 상담 해드려요""",한국아이닷컴 인기기사\n\n기업은행이 개인 고객들의 가계부채 해결 도우미로 나선다....


In [65]:
%%time
frm = pd.date_range('2010-01','2018-01',freq='YS').strftime('%Y.%m.%d')
tom = pd.date_range('2010-12','2019-12',freq='Y').strftime('%Y.%m.%d')
news = []
for f,t in zip(frm,tom):
    news.append(naver_search_d('가계부채',f,t,page=1))

In [72]:
newsdf = pd.concat(news,axis=0)
newsdf.to_csv('o:/보낼파일함/news.csv',index=False)